In [ ]:
import numpy as np
import pandas as pd
import bokeh as bk
import os
import json
import panel as pn
import param

from bokeh.models.widgets.tables import NumberFormatter, BooleanFormatter

In [ ]:
pn.extension(sizing_mode="stretch_width")

In [ ]:
def read_in_chunks(file_object, chunk_size=1024):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data
        
def build_stats_list(json_file_values='', json_file_expected='', test='test_standard_cube'):
    with open(json_file_values) as file:
        js = json.loads(file.read())
        
    keys = []
    values = []
    expected = []

    for key, value in js[test]['im_stats_dict'].items():
        if isinstance(value, list):
            for v in value:
                keys.append(key)
                values.append(v)
        else:
            keys.append(key)
            values.append(value)
            
    with open(json_file_expected) as file:
        js = json.loads(file.read())
    
    for key, value in js[test]['exp_im_stats'].items():
        if isinstance(value[-1], list):
            for v in value[-1]:
                expected.append(v)
        else:
            expected.append(value[-1])
            
    
    
    return pd.DataFrame({
        'key': keys,
        'value': values,
        'expected': expected
    })

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'gray' if val < 0 else 'black'
    return 'color: %s' % color


def stats_table(json_file_values='', json_file_expected='', test='test_standard_cube'):
    
    df = build_stats_list(
        json_file_values='test_standard_cube_current_metrics_6.2.0.124.json', 
        json_file_expected='test_stk_alma_pipeline_imaging_exp_dicts.json', 
        test=test
    )
    
    df['pass'] = (abs((df.expected - df.value)/df.expected)) < 0.05
    
    bokeh_formatters = {
        'value': NumberFormatter(format='0.00000'),
        'expected': NumberFormatter(format='0.00000'),
        'pass': BooleanFormatter(),
    }
    
    pn.widgets.Tabulator.theme = 'site'
    
    #select = pn.widgets.MultiSelect(options=df.key.unique().tolist(), name='Filter', width=100)
    #select = pn.widgets.Select(
    #    name='Key select', 
    #    options=df.key.unique().tolist()
    #)
    
    table = pn.widgets.Tabulator(
        df, 
        layout='fit_data_stretch', 
        formatters=bokeh_formatters,
        groupby=['pass'],
        pagination='remote'
    )

#.style.applymap(
#        color_negative_red, 
#        subset=['pass']
#    ).set_properties(**{'color': '#f38494'}, subset=['value'])

    #table.add_filter(select, 'key')    
    
    return table

In [ ]:
#df = build_stats_list(
#    json_file_values='test_standard_cube_current_metrics_6.2.0.124.json', 
#    json_file_expected='test_stk_alma_pipeline_imaging_exp_dicts.json', 
#    test='test_standard_cube'
#)

In [ ]:
#data = df

In [ ]:
#class ReactiveTables(param.Parameterized):
#    
#    title = param.String(default='TClean Testing Statistics')
#    dataset = param.ObjectSelector(default='im_stats', objects=['im_stats', 'im_masks', 'im_image'])
#    stat = param.ObjectSelector(default='npts', objects=['all', 'rms_per_chan', 'npts'])
#    rows = param.Integer(default=10, bounds=(0, 550))
    
#    @param.depends('dataset', 'stat')
#    def data(self):
#        if self.stat == 'all':
#            return data
#        else:
#            return data[data['keys']==self.stat]

#    @param.depends('data')
#    def summary(self):
#        return self.data().describe()
    
#    @param.depends('title')
#    def header(self):
#        return '##' + self.title
    
#    @param.depends('data', 'rows')
#    def table(self):
#        return self.data().iloc[:self.rows]
    
#    def panel(self):
#        return pn.Row(
#            pn.Param(self, name="Settings", width=200, sizing_mode="fixed"),
#            pn.Column(self.summary, self.table),
#            min_height=1000)

In [ ]:
#reactive = ReactiveTables()
#reactive.panel().servable()

In [ ]:
df = stats_table(
    json_file_values='test_standard_cube_current_metrics_6.2.0.124.json', 
    json_file_expected='test_stk_alma_pipeline_imaging_exp_dicts.json', 
    test='test_standard_cube'
)

In [ ]:
df

In [ ]:
import stats_table as st

In [ ]:
stats = st.StatsTable(
    json_measured='test_standard_cube_current_metrics_6.2.0.124.json', 
    json_expected='test_stk_alma_pipeline_imaging_exp_dicts.json' 
)

In [ ]:
stats.stats_table()